In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import os

tile=(-9,-18) #'-8_-18' #'5_-37'

os.environ['tile']= '%d_%d' % tile

In [ ]:
%%bash

ls_s3() {
  local prefix=$1
  aws s3 ls "${prefix}" | awk "{print \"$prefix\""'$4}'
}

vsis3_filter(){
  sed 's/^s3:\//\/vsis3/g'
}


base="dea-public-data/bench-data/LS5_TM_WATER"
tile="${tile:-'-8_-18'}"


dump="s3_tile_zip_${tile}.txt"


[ -e "${dump}" ] && echo "Tile listing exists, exiting early: ${tile}" && exit 0

ls_s3 "s3://${base}/${tile}/" > "$dump"

head -n 3 "$dump"
echo '...'
tail -n 2 "$dump"

In [ ]:
import rasterio
import numpy as np

In [ ]:
with open('s3_tile_zip_%d_%d.txt'%tile,'r') as f:
    files = [s.rstrip() for s in f.readlines()]
    
files[:3], len(files)

In [ ]:
f = rasterio.open(files[6])
print(f.bounds, f.shape)

bsz = [(i, j, f.block_size(1, i,j)) for j in range(16) for i in range(16)]

bsz = sorted(bsz, key=lambda s: s[-1], reverse=True)

im = f.read(1)

plt.imshow(im)
plt.colorbar()
bsz[:10]



In [ ]:
b = np.ndarray(f.block_shapes[0], dtype=f.dtypes[0])

fig = plt.figure(figsize=(15,3))

for i in range(5):
    f.read(1, window=f.block_window(1, *(bsz[i][:2])), out=b)
    ax = fig.add_subplot(1,5,i+1)
    ax.set_title('block: %d,%d'%(bsz[i][:2]))
    ax.imshow(b.copy(), vmin=0, vmax=255)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
fig.tight_layout()
    
bsz[:5]

In [ ]:
!ls s3_tile*

In [ ]:
f.compression